In [125]:
# Import packages #
import pandas as pd
import os
import lyricsgenius
from PyLyrics import *

In [126]:
os.getcwd()

'C:\\Users\\Rune-\\OneDrive - Aarhus universitet\\Oecon\\8. semester\\Indeng 135\\New project'

In [127]:
os.chdir('C:\\Users\\Rune-\\OneDrive - Aarhus universitet\\Oecon\\8. semester\\Indeng 135\\New project')

In [128]:
df = pd.read_csv('Data\\Spotify Data - Sheet1.csv')

In [129]:
df = df.drop(columns = ['Unnamed: 0', 'sel', 'Unnamed: 2'])

In [130]:
df.head()

,title,artist,top genre,year,added,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,It's You - Radio Edit,Syn Cole,big room,2015,2017‑10‑07,128,83,65,-4,13,49,212,11,5,58
1,Show You Love,Kato,danish hip hop,2017,2018‑10‑25,124,83,69,-5,7,66,183,29,7,47
2,Just Us,Said the Sky,chillstep,2018,2019‑02‑28,109,58,75,-5,10,33,163,19,6,68
3,Warriors - Rob Adans Remix,Nicky Romero,big room,2015,2017‑05‑17,127,89,67,-3,66,65,312,0,4,36
4,The Sweet Escape,Gwen Stefani,australian pop,2006,2016‑01‑08,120,77,75,-3,16,74,246,21,3,66


# Delete irrelevant rows #

In [131]:
df = df.drop_duplicates(subset=['title', 'artist'])
df = df[df.title.notnull()]
df = df[df.artist.notnull()]

In [132]:
df.shape

(1414, 15)

In [139]:
titles = []

# Change song names to only the core, such that you remove things as "Radio edit" and so on #
for title in df['title']:
    title = title.split(" - ")[0]
    title = title.split("(")[0]
    titles.append(title)
    
df['title_core'] = titles

# Get lyrics using lyricsgenius package 

In [142]:
genius = lyricsgenius.Genius("Da_pLO6QKdtfZ7QRnNaXdhmsxxtjRzNwwGJLWHKEHgxNM81GfF7oy5THRWeKMfTa") # Get access to Genius API #
genius.verbose = False # Turn off status messages

In [143]:
lyrics = []
i = 1
Fails = 0
for song, artist in zip(df['title_core'], df['artist']):
        try:
            new_song = genius.search_song(song, artist)
            lyrics.append(new_song.lyrics)
            i += 1
        except AttributeError:
            lyrics.append('NaN')
            print('Error with song %s:' % i)
            print(song, artist)
            print('Number of fails = %s' % Fails)
            print('\n')
            i += 1
            Fails += 1

Error with:
None Axwell /\ Ingrosso
Error with:
None Otto Knows
Error with:
None Dzeko & Torres
Error with:
None Madison Mars
Error with:
None Julian Calor
Error with:
None Kryder
Error with:
None Creedence Clearwater Revival
Error with:
None The Echelon Effect
Error with:
None Dario Marianelli
Error with:
None Dario Marianelli
Error with:
None Dario Marianelli
Error with:
None Balmorhea
Error with:
None The Echelon Effect
Error with:
None Balmorhea
Error with:
None Keith Kenniff
Error with:
None U137
Error with:
None The Echelon Effect
Error with:
None Tracey Chattaway
Error with:
None Lowercase Noises
Error with:
None Tracey Chattaway
Error with:
None James Horner
Error with:
None Mychael Danna
Error with:
None Michael Kamen
Error with:
None Tom Tykwer
Error with:
None James Newton Howard
Error with:
None Nicholas Britell
Error with:
None Ramin Djawadi
Error with:
None James Newton Howard
Error with:
None Hans Zimmer
Error with:
None Joey Pecoraro
Error with:
None Mad Q
Error with:
N

TypeError: 'NoneType' object is not subscriptable

In [146]:
for i in range(0, 1414 - 1307):
    lyrics.append('NaN')

In [147]:
df['lyrics'] = lyrics

# Sentiment analysis on the lyrics #

In [150]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import nltk
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rune-\AppData\Roaming\nltk_data...


True

In [151]:
vader = SentimentIntensityAnalyzer()
def vader_polarity(text):
    """ Transform the output to a binary 0/1 result """
    score = vader.polarity_scores(text)
    return 1 if score['pos'] > score['neg'] else 0

In [152]:
pos_score = []
neg_score = []
sent_score = []
for lyric in lyrics:
    score = vader.polarity_scores(lyric)
    pos_score.append(score['pos'])
    neg_score.append(score['neg'])
    sent_score.append(1*(score['pos'] > score['neg']))

In [153]:
df['lyric_pos_sent'] = pos_score
df['lyric_neg_sent'] = neg_score
df['Pos/Neg'] = sent_score

# Save new dataframe

In [160]:
df.to_csv('Spotify_v5.csv', sep='\t', encoding='utf-8')